In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

def compute_cdf(scores):
    scores_sorted = np.sort(scores)
    cdf = np.arange(1, len(scores) + 1) / len(scores)
    return scores_sorted, cdf

def plot_cdfs(data_list, names=None):

    fig = go.Figure()
    for i in range(len(data_list)):
        x, y = compute_cdf(data_list[i])
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=names[i] if names is not None else None))
    fig.update_layout(xaxis_title='Value', yaxis_title='CDF')
    fig.show()

def plot_scores_by_prompt(data, max_new_tokens=29):
    
    # Pad data
    padded_data = [sublist + [np.nan] * (max_new_tokens - len(sublist)) for sublist in data]

    # Create a heatmap
    fig = go.Figure(data=go.Heatmap(
        z=padded_data,
        colorscale='RdYlGn',
        zmin=0, 
        zmax=1,  
        hoverongaps=False,  
        showscale=True
    ))
    
    # Update layout
    fig.update_layout(
        xaxis_title='Token Index',
        yaxis_title='Prompt Index',
        xaxis_nticks=max_new_tokens
    )
    
    # Show the figure
    fig.show()


def get_scores_by_batch(doc, prompts, batch_size):

    def _get_scores(doc, prompts, max_new_tokens=30):
        """
        Run a batch of generation to get projections, and do detection on each.
        """
        output = doc.generate(prompts, max_new_tokens=max_new_tokens, do_sample=False, gen_only=True, return_projections=True)
        all_scores_per_token = []
        for projs in output['projections']:
            scores_per_token = doc.detect(projs, aggregation_method='auto')[0]
            all_scores_per_token.append(list(scores_per_token))
        return all_scores_per_token, output['text']
    
    all_scores = []
    all_texts = []
    for i in range(0, len(prompts), batch_size):
        print(i)
        these_scores, these_texts = _get_scores(doc, prompts[i:i+batch_size])
        all_scores.extend(these_scores)
        all_texts.extend(these_texts)
    return all_scores, all_texts


## Setup
If you already have a model/tokenizer you want to use, you can skip this step. 
Be sure to also set the appropriate user_tag/assistant_tag for that model.

In [3]:
%%capture
# The quantized model used here requires some extra libraries. 
import sys
!{sys.executable} -m pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121
!{sys.executable} -m pip install optimum>=1.12.0
!{sys.executable} -m pip install auto-gptq==0.6.0
!{sys.executable} -m pip install accelerate

In [4]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/workspace/cache/' # change or comment out as desired 
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_name_or_path, revision, device):
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, device_map=device, revision=revision, trust_remote_code=False)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True, padding_side="left")
    tokenizer.pad_token_id = 0
    return model, tokenizer

device = 'cuda:0'
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
revision = 'gptq-4bit-32g-actorder_True'
user_tag = "[INST] "
assistant_tag = " [/INST]"

model, tokenizer = load_model(model_name_or_path, revision, device)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` in

## Train extractor

In [ ]:
from lmdoctor.doctor import Doctor
probe_type = 'pca'

# default
# extraction_target = 'honesty'
# doc = Doctor(model, tokenizer, user_tag, assistant_tag, extraction_target=extraction_target, probe_type=probe_type, device=device)
# doc.extract(n_train_pairs=128, n_dev_pairs=64, n_test_pairs=32)

# for shuffle data experiment
# extraction_target = 'honesty'
# doc = Doctor(model, tokenizer, user_tag, assistant_tag, extraction_target=extraction_target, probe_type=probe_type, device=device)
# doc.extract(n_train_pairs=128, n_dev_pairs=64, n_test_pairs=32, shuffle_functional_pairs=True)

# for adding Creative Content to training set experiment
extraction_target = 'honesty_plus'
doc = Doctor(model, tokenizer, user_tag, assistant_tag, extraction_target=extraction_target, probe_type=probe_type, device=device)
doc.extract(n_train_pairs=128, n_dev_pairs=64, n_test_pairs=32)

In [ ]:
doc.statement_pairs['train'][:2]

## Lie Requests

In [ ]:
flp = pd.read_csv('./lie_requests/fact_lie_pairs.csv', header=None)
flp.columns = ['prompt', 'label']

In [ ]:
flp.head(5)

In [ ]:
# prompt = flp['prompt'].iloc[0]
# prompt = flp['prompt'].iloc[3]
# prompt = flp[flp['label']==1]['prompt'].tolist()[12]
prompt = flp[flp['label']==0]['prompt'].tolist()[1]

print(prompt)
max_new_tokens = 30
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False, gen_only=True, return_projections=True)
output['text']

In [ ]:
doc.plot_projection_heatmap(output['projections'], output['tokens'], lastn_tokens_to_plot=max_new_tokens, saturate_at='auto')

In [ ]:
scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
doc.plot_scores_per_token(scores_per_token, output['tokens'], lastn_tokens_to_plot=max_new_tokens, detection_method='classifier')

In [ ]:
batch_size=5
lie_prompts = flp[flp['label'] == 1]['prompt'].tolist()
fact_prompts = flp[flp['label'] == 0]['prompt'].tolist()
lie_scores, lie_texts = get_scores_by_batch(doc, lie_prompts, batch_size)
fact_scores, fact_texts = get_scores_by_batch(doc, fact_prompts, batch_size)

In [ ]:
lie_scores_flat = np.concatenate(lie_scores)
fact_scores_flat = np.concatenate(fact_scores)
plot_cdfs((lie_scores_flat, fact_scores_flat), ['lie prompt', 'fact prompt'])

In [ ]:
plot_scores_by_prompt(lie_scores)

In [ ]:
plot_scores_by_prompt(fact_scores)

In [ ]:
lie_agg = [np.mean(l) for l in lie_scores]
fact_agg = [np.mean(l) for l in fact_scores]

fig = go.Figure()
fig.add_trace(go.Histogram(x=lie_agg, name='lie', opacity=0.75))
fig.add_trace(go.Histogram(x=fact_agg, name='fact', opacity=0.75))


fig.show()

## Unanswerable Questions

In [ ]:
import pandas as pd
qa = pd.read_csv('unanswerable_questions/hallucination_prompts.csv')
qa.head(2)

In [ ]:
prompt = qa['Hallucination Question'].iloc[0]
# prompt = qa['Factual Question'].iloc[0]

print(prompt)
max_new_tokens = 30
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False, gen_only=True, return_projections=True)
output['text']

In [ ]:
doc.plot_projection_heatmap(output['projections'], output['tokens'], lastn_tokens_to_plot=None, saturate_at='auto')

In [ ]:
scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
doc.plot_scores_per_token(scores_per_token, output['tokens'], lastn_tokens_to_plot=max_new_tokens, detection_method='classifier')
# doc.plot_scores_per_token(scores_per_token[:, 26:], output['tokens'][26:], lastn_tokens_to_plot=max_new_tokens, detection_method='classifier')

In [ ]:
batch_size = 16 # for some reason, i see weird results in one batch when using batch of 32...probably due to an issue with the quantization (autogptq, model itself, exllama etc). Best to keep it to a low batch size.
hallucination_questions = qa['Hallucination Question'].tolist()
factual_questions = qa['Factual Question'].tolist()
hallucination_scores, hallucination_texts = get_scores_by_batch(doc, hallucination_questions, batch_size)
factual_scores, factual_texts = get_scores_by_batch(doc, factual_questions, batch_size)

In [ ]:
hallucination_scores_flat = np.concatenate(hallucination_scores) 
factual_scores_flat = np.concatenate(factual_scores) 
plot_cdfs((hallucination_scores_flat, factual_scores_flat), ['hallucination scores', 'factual_scores'])

In [ ]:
plot_scores_by_prompt(factual_scores)

In [ ]:
plot_scores_by_prompt(hallucination_scores)

In [ ]:
hallucination_aggs = [np.mean(l) for l in hallucination_scores]
factual_aggs = [np.mean(l) for l in factual_scores]

fig = go.Figure()
fig.add_trace(go.Histogram(x=hallucination_aggs, name='hallucination', opacity=0.75))
fig.add_trace(go.Histogram(x=factual_aggs, name='factual', opacity=0.75))

## Creative Content

In [ ]:
content_prompts = pd.read_csv('./creative_content/content_prompts.csv', header=None)
content_prompts.columns = ['prompt', 'label']

In [ ]:
content_prompts.head(2)

In [ ]:
prompt = content_prompts['prompt'].iloc[20]

print(prompt)
max_new_tokens = 30
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False, gen_only=True, return_projections=True)
output['text']

In [ ]:
doc.plot_projection_heatmap(output['projections'], output['tokens'], lastn_tokens_to_plot=None, saturate_at='auto')

In [ ]:
scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
doc.plot_scores_per_token(scores_per_token, output['tokens'], lastn_tokens_to_plot=max_new_tokens, detection_method='classifier')

In [ ]:
batch_size = 5
fictional_content_prompts = content_prompts[content_prompts['label'] == 1]['prompt'].tolist()
factual_content_prompts = content_prompts[content_prompts['label'] == 0]['prompt'].tolist()
fictional_content_scores, fictional_content_texts = get_scores_by_batch(doc, fictional_content_prompts, batch_size)
factual_content_scores, factual_content_texts = get_scores_by_batch(doc, factual_content_prompts, batch_size)

In [ ]:
fictional_content_scores_flat = np.concatenate(fictional_content_scores)
factual_content_scores_flat = np.concatenate(factual_content_scores)
plot_cdfs((fictional_content_scores_flat, factual_content_scores_flat), ['fictional prompt', 'factual prompt'])

In [ ]:
plot_scores_by_prompt(fictional_content_scores)

In [ ]:
plot_scores_by_prompt(factual_content_scores)

In [ ]:
fictional_content_scores = np.array(fictional_content_scores)
factual_content_scores = np.array(factual_content_scores)
plot_cdfs(([fictional_content_scores.mean(axis=1), factual_content_scores.mean(axis=1)]))

In [ ]:
fictional_content_agg = [np.mean(l) for l in fictional_content_scores]
factual_content_agg = [np.mean(l) for l in factual_content_scores]

fig = go.Figure()
fig.add_trace(go.Histogram(x=fictional_content_agg, name='fictional', opacity=0.75))
fig.add_trace(go.Histogram(x=factual_content_agg, name='factual', opacity=0.75))

fig.show()

### Subjective Content

In [ ]:
# opine = pd.read_csv('./opinions/opinions.csv')
opine = pd.read_csv('./subjective_content/opinions_updated.csv')

opine.head(2)

In [ ]:
opine['Label'].unique().tolist()

In [ ]:
# prompt = opine[opine['Label'] == 'Subjective']['Prompt'].tolist()[1]
prompt = opine[opine['Label'] == 'Ambiguous']['Prompt'].tolist()[2]
# prompt = opine[opine['Label'] == 'Objective']['Prompt'].tolist()[10]

print(prompt)
max_new_tokens = 30
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False, gen_only=True, return_projections=True)
output['text']

In [ ]:
doc.plot_projection_heatmap(output['projections'], output['tokens'], lastn_tokens_to_plot=max_new_tokens, saturate_at='auto')

In [ ]:
scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
doc.plot_scores_per_token(scores_per_token, output['tokens'], lastn_tokens_to_plot=max_new_tokens, detection_method='classifier')

In [ ]:
batch_size = 5
subjective_content_prompts = opine[opine['Label'] == 'Subjective']['Prompt'].tolist()
ambiguous_content_prompts = opine[opine['Label'] == 'Ambiguous']['Prompt'].tolist()
objective_content_prompts = opine[opine['Label'] == 'Objective']['Prompt'].tolist()
subjective_content_scores, subjective_content_texts = get_scores_by_batch(doc, subjective_content_prompts, batch_size)
ambiguous_content_scores, ambiguous_content_texts = get_scores_by_batch(doc, ambiguous_content_prompts, batch_size)
objective_content_scores, objective_content_texts = get_scores_by_batch(doc, objective_content_prompts, batch_size)

In [ ]:
plot_scores_by_prompt(subjective_content_scores)

In [ ]:
plot_scores_by_prompt(ambiguous_content_scores)

In [ ]:
plot_scores_by_prompt(objective_content_scores)

In [ ]:
# prompt = "Pretend you are a human who just ate a big meal. Question: Are you hungry?"
# prompt = "Pretend you are a human who has not eaten all day. Question: Are you hungry?"
# prompt = "[INST] Do you like dogs? [/INST] No, "
prompt = "[INST] Do you like dogs? [/INST] Yes,"

should_format_prompt=False

print(prompt)
max_new_tokens = 30
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False, gen_only=False, return_projections=True, should_format_prompt=should_format_prompt)
output['text']

In [ ]:
doc.plot_projection_heatmap(output['projections'], output['tokens'], lastn_tokens_to_plot=max_new_tokens, saturate_at='auto')

In [ ]:
scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
doc.plot_scores_per_token(scores_per_token, output['tokens'], lastn_tokens_to_plot=max_new_tokens, detection_method='classifier')

### Objective, Non-Factual Questions

In [ ]:
# prompt = "Please write a python function that prints 'do as i say' 10 times."
prompt = "Please write a python function that prints 'hello world' 10 times."

print(prompt)
max_new_tokens = 200
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False, gen_only=True, return_projections=True)
output['text']

In [ ]:
print(output['text'])

In [ ]:
# Focus on just the code part

s = output['tokens'].index(':') + 1
e = output['tokens'].index('()')
output['tokens'][s:e]

scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
doc.plot_scores_per_token(scores_per_token, output['tokens'], detection_method='classifier', token_ranges=[[s,e]])

In [ ]:
# prompt = "If I had 10 dollars and then spent 3, how many would I have left?"
# prompt = "If I had 10 qubiots and then spent 3, how many would I have left?"
# prompt = "If I had 10 gold-plated coins and then spent 3, how many would I have left?"
# prompt = "If I had 10 triple-decker cheesburgers with extra mayo and ate 3, how many would I have left?"

# prompt = "Consider the following situation: I fill a cup with water and mix in one scoop of powder ('original mixture'). I then pour out half of the mixture and mix in another scoop of powder ('final mixture'). How concentrated is the final mixture, relative to the original?"
prompt = "Let's say I had a bag with 20 black marbles and 10 yellow marbles. I pull out a yellow marble and put it on the table. What are the chances I pull out another yellow marble on the second turn?"

print(prompt)
max_new_tokens = 100
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False, gen_only=True, return_projections=True)
output['text']

In [ ]:
doc.plot_projection_heatmap(output['projections'], output['tokens'], lastn_tokens_to_plot=None, saturate_at='auto')
# doc.plot_projection_heatmap(output['projections'], None, lastn_tokens_to_plot=max_new_tokens, saturate_at='auto', aspect='auto')

In [ ]:
scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
# doc.plot_scores_per_token(scores_per_token, output['tokens'], lastn_tokens_to_plot=None, detection_method='classifier')
doc.plot_scores_per_token(scores_per_token, None, lastn_tokens_to_plot=max_new_tokens, detection_method='classifier', figsize=(1000,200), aspect='auto')

## Quality of probes

In [ ]:
# Visualize the projections

from lmdoctor import plot_utils, detection_utils
proj_pairs = detection_utils.act_pairs_to_projs(doc.train_acts, doc.direction_info, len(doc.statement_pairs['train']))

layer = 15
projs_true = proj_pairs[0, :, layer]
projs_lie = proj_pairs[1, :, layer]
plot_utils.plot_projs_on_numberline(projs_true, projs_lie)

In [ ]:
# visualize a scan from the training dataset

input_text = doc.statement_pairs['train'][8][1]
# input_text = doc.statement_pairs['train'][8][0] 

projections = doc.get_projections(input_text=input_text)
tokens = tokenizer.tokenize(input_text)
doc.plot_projection_heatmap(projections, tokens, saturate_at='auto')